In [3]:
import pandas as pd
import numpy as np
import akshare as ak
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
INIT_CAPITAL = 1e6  # 初始资金
FEE_RATE = 0.002  # 单边交易费率(含佣金税费)
TRADE_RATIO = 1.0  # 仓位比例

In [ ]:
COMPOUND = True  # 是否复利

In [4]:
# 策略参数扫描空间
BUY_TIMES = ['close']  # 买入时机选择
SELL_TIMES = ['open', 'open_1h', 'close']  # 卖出时机选择 TODO: 实现小时级别调整

# 加载预测数据
pred_df = pd.read_csv('score.csv', header=None, names=['date', 'score'])
pred_df['date'] = pd.to_datetime(pred_df['date'], format='%Y%m%d')
print(pred_df)

           date    score
0    2020-01-02  1266.02
1    2020-01-03   984.60
2    2020-01-06   567.40
3    2020-01-07   771.60
4    2020-01-08   364.80
...         ...      ...
1238 2025-02-17   561.05
1239 2025-02-18   217.32
1240 2025-02-19   903.43
1241 2025-02-20  -263.44
1242 2025-02-21  -328.53

[1243 rows x 2 columns]


In [16]:
import pandas as pd
import yfinance as yf

# 设置ETF代码和时间范围（最多获取最近60天的30分钟数据）
etf_code = "162411.SZ"  # 南方中证1000ETF示例代码，请确认实际代码
start_date = "2020-01-02"
end_date = "2025-02-21"

# 下载30分钟级别数据
data = yf.download(
    tickers=etf_code,
    start=start_date,
    end=end_date,
    interval="30m",
    progress=True
)

# 转换为Pandas DataFrame
df = pd.DataFrame(data)

# 检查数据
if df.empty:
    print("未获取到数据，请检查代码或时间范围！")
else:
    # 处理时区（可选：转换为无时区或本地时间）
    df.index = df.index.tz_localize(None)
    print("数据示例：")
    print(df.head())

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['162411.SZ']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


未获取到数据，请检查代码或时间范围！


In [32]:
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta

def get_minute_data(symbol, start_date, end_date, period='30'):
    """
    Get minute-level data for Chinese stocks/ETFs using akshare

    Parameters:
    - symbol: stock/ETF code (e.g., '162411' for 南方中证1000ETF)
    - start_date: start date in 'YYYYMMDD' format
    - end_date: end date in 'YYYYMMDD' format
    - period: minute interval ('1', '5', '15', '30', '60')

    Returns:
    - DataFrame with minute-level data
    """
    try:
        # Convert dates to required format if needed
        if isinstance(start_date, str) and '-' in start_date:
            start_date = start_date.replace('-', '')
        if isinstance(end_date, str) and '-' in end_date:
            end_date = end_date.replace('-', '')

        # Get minute-level data
        df = ak.stock_zh_a_minute(
            symbol=symbol,
            period=period,
            adjust='qfq'  # 前复权
        )

        # Convert datetime string to proper datetime object
        #                 day  open  high   low  close  volume
        # 2020-06-17 14:49:00  3.05  3.05  3.04   3.04  133200

        df['datetime'] = pd.to_datetime(df['day'])
        df.set_index('datetime', inplace=True)

        # # Rename columns to standard names
        # df.rename(columns={
        #     '开盘': 'Open',
        #     '收盘': 'Close',
        #     '最高': 'High',
        #     '最低': 'Low',
        #     '成交量': 'Volume',
        #     '成交额': 'Amount'
        # }, inplace=True)

        return df
    except Exception as e:
        print(f"Error fetching minute data: {e}")
        return pd.DataFrame()

# Example usage
symbol = 'sh512100'  # 南方中证1000ETF的代码
start_date = '20200101'  # Format: YYYYMMDD
end_date = '20250225'    # Format: YYYYMMDD

# Get 30-minute data
minute_data = get_minute_data(symbol, start_date, end_date, period='30')
print(minute_data.head())

                                     day   open   high    low  close  \
datetime                                                               
2024-02-28 14:30:00  2024-02-28 14:30:00  2.124  2.128  2.102  2.113   
2024-02-28 15:00:00  2024-02-28 15:00:00  2.113  2.120  2.066  2.069   
2024-02-29 10:00:00  2024-02-29 10:00:00  2.057  2.119  2.053  2.117   
2024-02-29 10:30:00  2024-02-29 10:30:00  2.118  2.136  2.114  2.126   
2024-02-29 11:00:00  2024-02-29 11:00:00  2.126  2.142  2.118  2.138   

                        volume  
datetime                        
2024-02-28 14:30:00  110747376  
2024-02-28 15:00:00  360936471  
2024-02-29 10:00:00  217190294  
2024-02-29 10:30:00   84043116  
2024-02-29 11:00:00   82124372  


In [21]:
etf = ak.fund_etf_category_sina(symbol="ETF基金")

In [26]:
df_500 = ak.fund_etf_hist_sina(symbol="sh510500")
df_1000 = ak.fund_etf_hist_sina(symbol="sh512100")  # 中证1000 day

In [28]:
df_1000

AttributeError: 'Index' object has no attribute '_format_flat'

            date   open   high    low  close      volume
0     2016-11-04  1.002  1.007  0.998  1.000   223992300
1     2016-11-07  0.999  1.003  0.996  0.998    23446900
2     2016-11-08  0.998  1.007  0.998  1.005    51134300
3     2016-11-09  1.004  1.006  0.980  0.998    12873300
4     2016-11-10  1.003  1.013  1.003  1.012    42887000
...          ...    ...    ...    ...    ...         ...
2017  2025-02-27  2.608  2.619  2.560  2.600   497368873
2018  2025-02-28  2.586  2.589  2.503  2.511  1270109921
2019  2025-03-03  2.514  2.552  2.496  2.508   650157255
2020  2025-03-04  2.492  2.545  2.490  2.541   559809503
2021  2025-03-05  2.540  2.559  2.524  2.558   565835320

[2022 rows x 6 columns]